### <p style="text-align: right;"> &#9989; **Context Matters** </p>
#### <p style="text-align: right;"> &#9989; Christina, Hao, Yunfei, Erik</p>

# Module 3

## How do gene transcription relate to phenotype?

We will work with maize.

### Preliminaries

First, import the usual libraries
- `math`: basic math operations
- `os`: enable file manipulation with the OS
- `sys`: enable interaction with commandline
- `importlib` : reload libraries if necessary
- `argparse` : pass arguments directly to the command line
- `glob`: more variable manipulation
- `matplotlib.pyplot`: default plotter (I personally like ggplot waaaaay better. E)
    - `inline`: so that plots are shown in the notebook
- `numpy`: all number cruching done here
- `pandas`: data wrangling

In [1]:
import math
import importlib
import numpy as np
import os
import sys
import argparse
import glob
from matplotlib import pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import seaborn as sns
sns.set(color_codes=True)

Specify the gene expression raw data file and a directory to save plots and CSVs.

The eQTL file was provided by (Christina,Fabio,Nolan,Scott)-group in Module 1. Supposedly, it is the result of eQTL performed of every gene against every SNP.

In [4]:
# path to FPKM data
gene_file = '~/documents/css893/942_FPKM_B73_genes_w_feature.txt'

# path to eQTL data
eqtl_file = '/home/ejam/documents/css893/Merged_eQTL_Data.tsv'

# path to correlation matrix directory
src = '/home/ejam/documents/css893/tpm_corr/'

# path to save results
dst = '/home/ejam/documents/css893/context_matters/results/'

Load the original FPKM data. This time we only care of the metadata. All the contigs were treated as _chromosome 11_

In [5]:
fpkm = pd.read_table(gene_file)
metadata_cols = 5

gene_info = fpkm.iloc[:, :metadata_cols]
foo = gene_info['chromosome'].unique()

for idx in foo:
    if idx == str(idx):
        value = 0
        if idx[0] == 'B':
            value = int('73' + (idx.split('_')[-1])[3:])
            value = 11
        else:
            value = int(idx)
        
        mask = gene_info.chromosome == idx
        gene_info.loc[mask, 'chromosome'] = value

/home/ejam/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Load the correlation matrix. The correlation method is _pearson_ and we only consider those genes whose TPM variance was above 10e5.

In [3]:
cutoff = 10000
corr_meth = 'pearson'
corr_csv = '{}_gene_correlation_{}.csv'.format(corr_meth,cutoff)

corr = pd.read_csv(src + corr_csv)
corr

,60,74,85,95,114,169,199,415,562,657,...,43720,43830,43873,44001,44033,44111,44119,44155,44221,44254
0,1.000000,-0.203803,-0.041840,0.135602,0.297998,-0.025113,-0.373148,0.335764,-0.295790,0.397197,...,-0.309133,-0.326422,-0.096837,-0.186167,0.136122,-0.417258,0.143673,-0.036973,0.028265,-0.382432
1,-0.203803,1.000000,0.084412,-0.320546,-0.071207,0.101266,0.424646,-0.175569,0.446181,-0.486499,...,0.386614,0.424727,0.153691,0.069929,-0.348278,0.370390,0.104439,-0.068258,0.202899,0.455165
2,-0.041840,0.084412,1.000000,-0.045254,-0.170664,-0.124421,0.323632,-0.062687,0.161443,-0.193569,...,0.360204,0.271762,0.036250,-0.038425,0.057585,0.115981,0.112563,0.080347,0.111763,0.141706
3,0.135602,-0.320546,-0.045254,1.000000,0.097013,0.056162,-0.387976,0.232699,-0.433461,0.467508,...,-0.401847,-0.356511,-0.150706,-0.066169,0.363566,-0.399549,-0.029409,0.056737,-0.233748,-0.494055
4,0.297998,-0.071207,-0.170664,0.097013,1.000000,0.076526,-0.309533,0.360927,-0.220001,0.279083,...,-0.307539,-0.251303,0.034189,-0.165472,-0.075354,-0.314652,0.182568,-0.121469,0.038715,-0.243578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643,-0.417258,0.370390,0.115981,-0.399549,-0.314652,0.030274,0.669776,-0.405258,0.643508,-0.785690,...,0.605081,0.549997,0.256985,0.361084,-0.449045,1.000000,-0.084839,-0.007476,0.207629,0.803752
644,0.143673,0.104439,0.112563,-0.029409,0.182568,0.087487,0.010110,0.133939,0.118709,-0.055325,...,0.083770,-0.110514,0.022980,-0.062397,-0.051659,-0.084839,1.000000,-0.097842,0.183584,0.012121
645,-0.036973,-0.068258,0.080347,0.056737,-0.121469,0.012606,0.072143,-0.181021,-0.051429,0.074363,...,0.234961,0.025535,-0.005260,0.291611,0.384691,-0.007476,-0.097842,1.000000,0.010586,-0.131849
646,0.028265,0.202899,0.111763,-0.233748,0.038715,0.088365,0.226196,-0.125008,0.300624,-0.298867,...,0.369234,0.205502,0.174752,0.181013,-0.252004,0.207629,0.183584,0.010586,1.000000,0.319675


Retrieve the metadata corresponding to the highly varying genes.

In [20]:
meta = gene_info.iloc[corr.columns]
foo = meta.chromosome.unique()
print(len(foo), meta.shape)
meta

11 (648, 5)


,gene,chromosome,feature_type,position_left,position_right
60,Zm00001d032825,1,gene,239383320,239384004
74,Zm00001d014957,5,gene,69975364,69978349
85,Zm00001d036567,6,gene,92808874,92811320
95,Zm00001d013039,5,gene,3603516,3604227
114,Zm00001d051174,4,gene,147173977,147176860
...,...,...,...,...,...
44111,Zm00001d021421,7,gene,151705668,151712216
44119,Zm00001d053593,4,gene,235759773,235760235
44155,Zm00001d027442,1,gene,5434307,5435871
44221,Zm00001d030725,1,gene,155959517,155963384


In [8]:
eqtl = pd.read_table(eqtl_file)
eqtl

,R_Gene_Chr,R_Gene_Start,R_Gene_Stop,R_Gene_ID,R_Gene_Class,SNP_Chr,SNP_Start,SNP_Stop,SNP_ID,Associated_Gene_ID,Statistic,PVal,FDR,Beta
0,chr1,44288,49837,Zm00001d027230,gene,chr1,44306,44307,rs1_44306,Zm00001d047861,-15.244080,4.703940e-47,8.784256e-43,-0.012608
1,chr1,44288,49837,Zm00001d027230,gene,chr1,44306,44307,rs1_44306,Zm00001d004808,-10.441780,3.181453e-24,2.339655e-20,-3.733139
2,chr1,44288,49837,Zm00001d027230,gene,chr1,44306,44307,rs1_44306,Zm00001d031538,-10.165090,4.220149e-23,2.880657e-19,-12.332500
3,chr1,44288,49837,Zm00001d027230,gene,chr1,44306,44307,rs1_44306,Zm00001d008253,-10.165090,4.220149e-23,2.880657e-19,-13.845900
4,chr1,44288,49837,Zm00001d027230,gene,chr1,44306,44307,rs1_44306,Zm00001d032862,-10.165090,4.220149e-23,2.880657e-19,-0.000578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17196037,chr9,159667997,159691083,Zm00001d048577,gene,chr9,159690349,159690350,rs9_159690349,Zm00001d001752,-7.556017,9.846873e-14,3.089405e-10,-0.006062
17196038,chr9,159667997,159691083,Zm00001d048577,gene,chr9,159690349,159690350,rs9_159690349,Zm00001d001652,-7.371548,3.694633e-13,1.089526e-09,-0.042778
17196039,chr9,159667997,159691083,Zm00001d048577,gene,chr9,159690349,159690350,rs9_159690349,Zm00001d045638,-7.322923,5.210988e-13,1.501189e-09,-6.382181
17196040,chr9,159667997,159691083,Zm00001d048577,gene,chr9,159690349,159690350,rs9_159690349,Zm00001d031869,-7.303307,5.983143e-13,1.712528e-09,-0.196164


In [22]:
eqtl.R_Gene_Chr.unique()

array(['chr1', 'chr10', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7',
       'chr8', 'chr9'], dtype=object)

### Making subsets of the eQTL data

We subsample the eQTL matrix above. 
+ Consider only the eQTLs where the SNP is located in one of the highly varying genes.
+ Consider only the eQTLs where the SNP is associated to one of the highly varying genes. 

In [26]:
i = 0
located = pd.DataFrame()

for i in range(meta.shape[0]):
    located = located.append(eqtl[ eqtl['R_Gene_ID'] == meta.iloc[i]['gene']] , sort=False)

located

,R_Gene_Chr,R_Gene_Start,R_Gene_Stop,R_Gene_ID,R_Gene_Class,SNP_Chr,SNP_Start,SNP_Stop,SNP_ID,Associated_Gene_ID,Statistic,PVal,FDR,Beta
1907240,chr1,239383320,239384004,Zm00001d032825,gene,chr1,239383364,239383365,rs1_239383364,Zm00001d015191,-13.667570,6.379975e-39,9.103020e-35,-0.027237
1907241,chr1,239383320,239384004,Zm00001d032825,gene,chr1,239383364,239383365,rs1_239383364,Zm00001d032949,-13.667570,6.379975e-39,9.103020e-35,-0.016791
1907242,chr1,239383320,239384004,Zm00001d032825,gene,chr1,239383364,239383365,rs1_239383364,Zm00001d044935,-13.667570,6.379975e-39,9.103020e-35,-0.001824
1907243,chr1,239383320,239384004,Zm00001d032825,gene,chr1,239383364,239383365,rs1_239383364,Zm00001d019017,-13.667570,6.379975e-39,9.103020e-35,-0.011985
1907244,chr1,239383320,239384004,Zm00001d032825,gene,chr1,239383364,239383365,rs1_239383364,Zm00001d042657,-13.667570,6.379975e-39,9.103020e-35,-0.020081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617571,chr1,41833737,41838204,Zm00001d028653,gene,chr1,41837857,41837858,rs1_41837857,Zm00001d019489,-7.031328,3.932666e-12,1.017631e-08,-0.006891
617572,chr1,41833737,41838204,Zm00001d028653,gene,chr1,41837857,41837858,rs1_41837857,Zm00001d001706,-6.838447,1.439989e-11,3.450679e-08,-0.048347
617573,chr1,41833737,41838204,Zm00001d028653,gene,chr1,41837857,41837858,rs1_41837857,Zm00001d030802,-6.737078,2.812868e-11,6.455689e-08,-0.129585
617574,chr1,41833737,41838204,Zm00001d028653,gene,chr1,41837857,41837858,rs1_41837857,Zm00001d033812,-6.726736,3.010252e-11,6.881629e-08,-0.006297


In [35]:
print(len(located.R_Gene_ID.unique()), 'genes out of', len(meta.gene.unique()),
      'contain associated SNPs')
located.drop(columns=['SNP_Chr','SNP_Stop','R_Gene_Class']).to_csv(dst+'eqtl_located_{}.csv'.format(cutoff),
                                                                  index = True, index_label='Original_Index')

639 genes out of 648 contain associated SNPs


In [23]:
related = pd.DataFrame()

for i in range(meta.shape[0]):
    related = related.append(eqtl[ eqtl['Associated_Gene_ID'] == meta.iloc[i]['gene']], sort=False)

related

,R_Gene_Chr,R_Gene_Start,R_Gene_Stop,R_Gene_ID,R_Gene_Class,SNP_Chr,SNP_Start,SNP_Stop,SNP_ID,Associated_Gene_ID,Statistic,PVal,FDR,Beta
46166,chr1,3095307,3095604,Zm00001d027332,gene,chr1,3095600,3095601,rs1_3095600,Zm00001d032825,-6.583017,7.653515e-11,1.648439e-07,-618.7353
89558,chr1,4494974,4499451,Zm00001d027403,gene,chr1,4495369,4495370,rs1_4495369,Zm00001d032825,-6.583017,7.653515e-11,1.648439e-07,-618.7353
95346,chr1,4891940,4894990,Zm00001d027419,gene,chr1,4892252,4892253,rs1_4892252,Zm00001d032825,-6.583017,7.653515e-11,1.648439e-07,-618.7353
122615,chr1,5819751,5825029,Zm00001d027462,gene,chr1,5824650,5824651,rs1_5824650,Zm00001d032825,-6.583017,7.653515e-11,1.648439e-07,-618.7353
142221,chr1,6407152,6409815,Zm00001d027488,gene,chr1,6409375,6409376,rs1_6409375,Zm00001d032825,-6.583017,7.653515e-11,1.648439e-07,-618.7353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15900382,chr8,178335675,178339115,Zm00001d012670,gene,chr8,178336921,178336922,rs8_178336921,Zm00001d013766,-9.052595,7.876414e-19,4.003547e-15,-816.8673
16337373,chr9,57489278,57491429,Zm00001d046036,gene,chr9,57489366,57489367,rs9_57489366,Zm00001d013766,-7.915426,6.914868e-15,2.463223e-11,-456.7894
17099343,chr9,155994046,155999070,Zm00001d048424,gene,chr9,155996738,155996739,rs9_155996738,Zm00001d013766,-7.062233,3.185022e-12,8.335277e-09,-915.2276
17160578,chr9,157608353,157609505,Zm00001d048513,gene,chr9,157609219,157609220,rs9_157609219,Zm00001d013766,-6.979505,5.590658e-12,1.419101e-08,-905.0377


In [36]:
print(len(related.Associated_Gene_ID.unique()), 'genes out of', len(meta.gene.unique()),
      'contain associated SNPs')
related.drop(columns=['SNP_Chr','SNP_Stop','R_Gene_Class']).to_csv(dst+'eqtl_related_{}.csv'.format(cutoff),
                                                                  index = True, index_label='Original_Index')

133 genes out of 648 contain associated SNPs


In [42]:
locrel = pd.concat([located, related], axis=1, join='inner')
locrel = locrel.iloc[:, :located.shape[1]]
locrel

,R_Gene_Chr,R_Gene_Start,R_Gene_Stop,R_Gene_ID,R_Gene_Class,SNP_Chr,SNP_Start,SNP_Stop,SNP_ID,Associated_Gene_ID,Statistic,PVal,FDR,Beta
10417261,chr5,69975364,69978349,Zm00001d014957,gene,chr5,69975582,69975583,rs5_69975582,Zm00001d040202,-7.155506,1.677400e-12,4.564204e-09,-580.0772
10417335,chr5,69975364,69978349,Zm00001d014957,gene,chr5,69975588,69975589,rs5_69975588,Zm00001d040202,-7.563082,9.355389e-14,2.941734e-10,-611.2713
10417910,chr5,69975364,69978349,Zm00001d014957,gene,chr5,69978178,69978179,rs5_69978178,Zm00001d040202,-7.563082,9.355389e-14,2.941734e-10,-611.2713
12014337,chr6,92808874,92811320,Zm00001d036567,gene,chr6,92811304,92811305,rs6_92811304,Zm00001d001139,-8.917276,2.446043e-18,1.199951e-14,-1262.5240
14255497,chr7,173953536,173954457,Zm00001d022264,gene,chr7,173954253,173954254,rs7_173954253,Zm00001d044495,-8.451948,1.081581e-16,4.604894e-13,-823.7607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616952,chr1,41833737,41838204,Zm00001d028653,gene,chr1,41836775,41836776,rs1_41836775,zma-MIR482,-8.789329,7.050030e-18,3.306002e-14,-288.7677
616999,chr1,41833737,41838204,Zm00001d028653,gene,chr1,41836824,41836825,rs1_41836824,zma-MIR482,-8.789329,7.050030e-18,3.306002e-14,-288.7677
617046,chr1,41833737,41838204,Zm00001d028653,gene,chr1,41836963,41836964,rs1_41836963,zma-MIR482,-8.789329,7.050030e-18,3.306002e-14,-288.7677
617075,chr1,41833737,41838204,Zm00001d028653,gene,chr1,41837097,41837098,rs1_41837097,zma-MIR482,-8.439968,1.189764e-16,5.001608e-13,-268.1414


In [47]:
print(len(locrel.Associated_Gene_ID.unique()), 'genes out of', len(meta.gene.unique()),
      'contain associated SNPs')
related.drop(columns=['SNP_Chr','SNP_Stop','R_Gene_Class']).to_csv(dst+'eqtl_locrel_{}.csv'.format(cutoff),
                                                                  index = True, index_label='Original_Index')

92 genes out of 648 contain associated SNPs
